In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import random
import torch.nn as nn
import torch.nn.functional as functional

In [9]:
def LoadImages(FileName):
    MyList = []
    BigImage = Image.open(FileName)
    # extraction de chaque vignette
    for x in range(0,BigImage.size[0],32):
        for y in range(0,BigImage.size[1],32):
            box = (x,y,x+32,y+32)
            Img = BigImage.crop(box)
            #if x+y == 0 : Img.show()

            # normalisation est stockage dans la liste
            Tensor = torchvision.transforms.functional.to_tensor(Img)
            MeanPerChannel = Tensor.mean((1,2))
            StdPerChannel  = Tensor.std((1,2))
            transform = transforms.Normalize(MeanPerChannel,StdPerChannel)
            Tensor = transform(Tensor)
            MyList.append(Tensor)
    return MyList

In [10]:
# creation des train and test sets
ListCatTensors  = LoadImages("res/cats.jpg")
ListDogTensors  = LoadImages("res/dogs.jpg")

In [29]:
# création du reseau

# tenseur des images : [ numéro, channels, LARG, HAUT ]
# param de la couche  CONV   : Conv2d(nb channels en entrée, nb de neurones, taille du kernel)

conv_A = nn.Conv2d(3, 6, 5)
pool   = nn.MaxPool2d(2)
conv_B = nn.Conv2d(6, 8, 5)
fc_A   = nn.Linear(8 * 5 * 5, 120)
fc_B   = nn.Linear(120, 84)
fc_C   = nn.Linear(84, 2)



In [12]:
def forward(x):
 
    print ("X        : ",x.shape)
    x = conv_A(x)
    print ("CONV2D_A : ",x.shape)
    x = functional.relu(x)
    print ("RELU     : ",x.shape)
    x = pool(x)
    print ("POOL     : ",x.shape)
    x = conv_B(x)
    print ("CONV2D_B : ",x.shape)
    x = functional.relu(x)
    print ("RELU     : ",x.shape)
    x = pool(x)
    print ("POOL     : ",x.shape)
    x = x.view(-1, 8 * 5 * 5)
    print ("VIEW     : ",x.shape)
    x = fc_A(x)
    print ("FC_A     : ",x.shape)
    x = functional.relu(x)
    print ("RELU     : ",x.shape)
    x = fc_B(x)
    print ("FC_B     : ",x.shape)
    x = functional.relu(x)
    print ("RELU     : ",x.shape)
    x = fc_C(x)
    print ("FC_C     : ",x.shape)
    return x

In [30]:
random.shuffle(ListCatTensors)
random.shuffle(ListDogTensors)

CatTrain = torch.stack(ListCatTensors[:5000])
CatTest  = torch.stack(ListCatTensors[5000:])

DogTrain = torch.stack( ListDogTensors[:5000] )
DogTest  = torch.stack( ListDogTensors[5000:] )

print(conv_A.weight.size())
print(fc_A.weight.size())



forward(DogTest)

torch.Size([6, 3, 5, 5])
torch.Size([120, 200])
X        :  torch.Size([1000, 3, 32, 32])
CONV2D_A :  torch.Size([1000, 6, 28, 28])
RELU     :  torch.Size([1000, 6, 28, 28])
POOL     :  torch.Size([1000, 6, 14, 14])
CONV2D_B :  torch.Size([1000, 8, 10, 10])
RELU     :  torch.Size([1000, 8, 10, 10])
POOL     :  torch.Size([1000, 8, 5, 5])
VIEW     :  torch.Size([1000, 200])
FC_A     :  torch.Size([1000, 120])
RELU     :  torch.Size([1000, 120])
FC_B     :  torch.Size([1000, 84])
RELU     :  torch.Size([1000, 84])
FC_C     :  torch.Size([1000, 2])


tensor([[-0.0418, -0.0969],
        [-0.0339, -0.0928],
        [-0.0212, -0.0936],
        ...,
        [-0.0298, -0.0824],
        [-0.0206, -0.0969],
        [-0.0471, -0.0882]], grad_fn=<AddmmBackward>)

Questions:

1. La couche Conv2DA produit un tenseur de taille : (6,28,28), pourquoi ?

    * Une entrée la couche Conv2DA est de taille $(3, 32, 32)$. La convolution est de noyau de taille $(5, 5)$, d'où $(32 - 5 + 1, 32 - 5 + 1) = (28, 28)$ pour une seule chaîne. On fixe le nombre de neurones à six six, qui est aussi le nombre de chaînes de sortie.

2. Relu ne modifie pas la taille du tenseur, pourquoi ?

    * La fonction ReLU est une fonction d'activation a pour but de rompre la linéarité entre couches pour éviter de factoriser le réseau en une seule multiplication matricielle, et ainsi conserver la complexité du réseau.Elle permet donc de faire suivre deux couches de mêmes dimensions.

3. Conv2D_B donne une taille 8x10x10, est-ce bien le résultat attendu ?

    * Cette fois la taille d'entrée est de $(6, 14, 14)$. Un noyau de taille $(5, 5)$ donne $(10, 10)$ pour une chaîne. La couche fait passer de six à huit chaînes, comme il était spécifié lors de la construction.

4. Quel est le nombre de poids de chaque couche ? (utilisez la propriété weight du tenseur)

5. Quel est la couche contenant le plus de poids ?

6. N'ayant que 5000 images en entrée, il faut éviter le surapprentissage, pour cela :
    a) supprimez la couche contenant le plus de poids
    b) divisez ~ par 2 le nombre de poids présents sur l'ensemble des couches FC¶
   
7. En début d'apprentissage, votre taux de réussite devrait etre proche de 50%, pourquoi ?

     * Les points sont initialisés avec des valeurs aléatoire, le modèle classifie donc en moyenne au hasard et n'a aucun intérêt tel quel (dans ce cas particulier d'une classification binaire, il serait plus utile d'avoir un taux de réussite proche de %0, il suffirait alors de prendre l'autre catégorie !)


In [ ]:
Développement :
    
    1. Ecrivez la passe forward pour les chats et les chiens
    2. Ecrivez la fonction calculant le LOSS total (vu en cours)
    3. Déclenchez la backpropagation
    4. Modifiez les poids de chaque couche (gradient)

    5. Ecrivez une fonction test() qui calcule les prédictions sur les échantillons de tests
        appel : les échantillons de tests ne doivent jamais etre utilisés pour le training
    6. Calculez ensuite le % de réussite de votre réseau sur cet échantillon


# Si les itérations sont trop lentes, vous pouvez calculer une estimation du gradient en utilisant
#    seulement 50 à 200 images prises au hasard dans le set de Train à chaque itération. Attention,
#    il faut dans tous les cas, prendre autant de chats que de chiens. De la recherche en ligne
#    sera surement necessaire

    7. Testez différentes valeurs pour le pas

# Affichez à chaque itération le loss ainsi que le taux de réussite